#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/competitions/desafio-de-prediccion-de-precios-de-portatiles

## Librerias

In [120]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import bootcampviztools as bt
import toolbox_ML as tb

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression,SelectFromModel, RFE, SequentialFeatureSelector
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)

## Data

In [78]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 
df = pd.read_csv("./data/train.csv")

## Data exploration

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                912 non-null    int64  
 1   laptop_ID         912 non-null    int64  
 2   Company           912 non-null    object 
 3   Product           912 non-null    object 
 4   TypeName          912 non-null    object 
 5   Inches            912 non-null    float64
 6   ScreenResolution  912 non-null    object 
 7   Cpu               912 non-null    object 
 8   Ram               912 non-null    object 
 9   Memory            912 non-null    object 
 10  Gpu               912 non-null    object 
 11  OpSys             912 non-null    object 
 12  Weight            912 non-null    object 
 13  Price_euros       912 non-null    float64
dtypes: float64(2), int64(2), object(10)
memory usage: 99.9+ KB


In [80]:
df.head(15)

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4GB,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00
3,578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37
5,867,376,Asus,ZenBook Flip,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,12GB,512GB SSD + 2TB HDD,Nvidia GeForce GT 940MX,Windows 10,2.26kg,1099.00
6,1136,60,Asus,X541UA-DM1897 (i3-6006U/4GB/256GB/FHD/Linux),Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,4GB,256GB SSD,Intel HD Graphics 520,Linux,2kg,415.00
7,609,1157,MSI,GP62M 7RDX,Gaming,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8GB,128GB SSD + 1TB HDD,Nvidia GeForce GTX 1050,Windows 10,2.4kg,1142.80
8,474,110,Asus,UX430UQ-GV209R (i7-7500U/8GB/256GB/GeForce,Ultrabook,14.0,IPS Panel Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,256GB SSD,Nvidia GeForce 940MX,Windows 10,1.3kg,1193.00
9,838,1263,Asus,Rog GL553VE-FY052T,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1050 Ti,Windows 10,2.5kg,1600.00


In [81]:
df.columns

Index(['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
       'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight',
       'Price_euros'],
      dtype='object')

In [82]:
df.Company.value_counts()

Company
Lenovo       208
Dell         207
HP           189
Asus         108
Acer          75
Toshiba       37
MSI           34
Apple         17
Mediacom       7
Razer          6
Samsung        6
Fujitsu        3
Microsoft      3
Vero           3
Xiaomi         3
Google         2
Chuwi          2
LG             1
Huawei         1
Name: count, dtype: int64

In [83]:
df.Product.value_counts()

Product
Inspiron 3567                                    21
XPS 13                                           20
250 G6                                           19
Vostro 3568                                      16
Inspiron 5570                                    11
Legion Y520-15IKBN                               11
ProBook 450                                      11
Inspiron 5567                                    10
Inspiron 7567                                    10
Satellite Pro                                    10
Aspire 3                                         10
Alienware 17                                     10
EliteBook 840                                     9
ThinkPad X1                                       9
XPS 15                                            8
MacBook Pro                                       8
EliteBook 850                                     8
Vostro 5568                                       7
ProBook 470                                       7
Idea

In [84]:
df[df['Product'] == 'X541NA-GO020T (N3350/4GB/1TB/W10)']

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
765,1183,1273,Asus,X541NA-GO020T (N3350/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Celeron Dual Core N3350 1.1GHz,4GB,1TB HDD,Intel HD Graphics 500,Windows 10,2kg,339.0


In [85]:
df.TypeName.value_counts()

TypeName
Notebook              522
Gaming                138
Ultrabook             135
2 in 1 Convertible     85
Workstation            18
Netbook                14
Name: count, dtype: int64

In [86]:
df.Inches.value_counts()

Inches
15.6    474
14.0    143
13.3    111
17.3    108
12.5     28
11.6     21
12.0      6
10.1      4
13.9      4
15.4      3
15.0      3
13.5      3
12.3      2
13.0      1
18.4      1
Name: count, dtype: int64

In [87]:
df.ScreenResolution.value_counts()

ScreenResolution
Full HD 1920x1080                                346
1366x768                                         208
IPS Panel Full HD 1920x1080                      163
IPS Panel Full HD / Touchscreen 1920x1080         37
Full HD / Touchscreen 1920x1080                   32
Touchscreen 1366x768                              15
1600x900                                          11
IPS Panel 4K Ultra HD 3840x2160                   10
4K Ultra HD / Touchscreen 3840x2160                8
Quad HD+ / Touchscreen 3200x1800                   8
IPS Panel 4K Ultra HD / Touchscreen 3840x2160      8
IPS Panel Retina Display 2304x1440                 6
IPS Panel Retina Display 2560x1600                 5
IPS Panel 1366x768                                 5
4K Ultra HD 3840x2160                              5
IPS Panel Touchscreen 1920x1200                    4
Touchscreen 2560x1440                              4
IPS Panel Quad HD+ / Touchscreen 3200x1800         4
IPS Panel Touchscreen 2560x14

In [88]:
df.Cpu.value_counts()

Cpu
Intel Core i5 7200U 2.5GHz               142
Intel Core i7 7700HQ 2.8GHz              102
Intel Core i7 7500U 2.7GHz                88
Intel Core i5 8250U 1.6GHz                54
Intel Core i5 6200U 2.3GHz                51
Intel Core i3 6006U 2GHz                  47
Intel Core i7 8550U 1.8GHz                44
Intel Core i7 6500U 2.5GHz                34
Intel Core i3 7100U 2.4GHz                29
Intel Core i7 6700HQ 2.6GHz               28
Intel Celeron Dual Core N3350 1.1GHz      22
Intel Celeron Dual Core N3060 1.6GHz      20
Intel Core i5 7300HQ 2.5GHz               19
Intel Pentium Quad Core N4200 1.1GHz      12
Intel Core i7 6600U 2.6GHz                12
Intel Pentium Quad Core N3710 1.6GHz      10
Intel Core i3 6006U 2.0GHz                10
Intel Core i7 7600U 2.8GHz                 9
Intel Core i5 6300U 2.4GHz                 9
Intel Celeron Dual Core N3050 1.6GHz       8
Intel Core i5 7300U 2.6GHz                 7
AMD A9-Series 9420 3GHz                    7
Intel 

In [89]:
df.Ram.value_counts()

Ram
8GB     434
4GB     274
16GB    125
6GB      32
2GB      16
12GB     15
32GB     14
24GB      2
Name: count, dtype: int64

In [90]:
df.Memory.value_counts()

Memory
256GB SSD                        293
1TB HDD                          150
500GB HDD                        100
512GB SSD                         75
128GB SSD +  1TB HDD              61
128GB SSD                         56
256GB SSD +  1TB HDD              52
32GB Flash Storage                27
64GB Flash Storage                11
2TB HDD                           11
512GB SSD +  1TB HDD              10
1TB SSD                            9
1.0TB Hybrid                       7
256GB Flash Storage                7
32GB SSD                           5
180GB SSD                          5
256GB SSD +  2TB HDD               4
16GB Flash Storage                 4
16GB SSD                           3
256GB SSD +  500GB HDD             2
1TB SSD +  1TB HDD                 2
128GB Flash Storage                2
512GB SSD +  2TB HDD               2
512GB Flash Storage                2
1.0TB HDD                          1
32GB HDD                           1
128GB SSD +  2TB HDD           

In [91]:
df.Gpu.value_counts()

Gpu
Intel HD Graphics 620             199
Intel HD Graphics 520             138
Intel UHD Graphics 620             45
Nvidia GeForce GTX 1050            43
Nvidia GeForce GTX 1060            31
Intel HD Graphics 400              28
AMD Radeon 530                     27
Intel HD Graphics 500              27
Nvidia GeForce 940MX               26
Nvidia GeForce GTX 1070            24
Intel HD Graphics                  23
Nvidia GeForce GTX 1050 Ti         21
AMD Radeon R5 M430                 18
Nvidia GeForce 930MX               16
Nvidia GeForce MX150               12
Intel HD Graphics 615              12
Nvidia GeForce 920MX               11
AMD Radeon 520                     11
Intel HD Graphics 505              11
Nvidia GeForce GTX 960M            10
Intel HD Graphics 515               9
AMD Radeon R7 M445                  9
AMD Radeon R5                       8
Nvidia Quadro M1200                 7
Intel HD Graphics 405               7
Intel Iris Plus Graphics 640        6
Nvidia G

In [92]:
df.OpSys.value_counts()

OpSys
Windows 10      746
Linux            47
No OS            44
Windows 7        32
Chrome OS        20
macOS            11
Mac OS X          6
Windows 10 S      4
Android           2
Name: count, dtype: int64

In [93]:
df.Weight.value_counts()

Weight
2.2kg      86
2.1kg      40
2.3kg      35
2.4kg      31
2kg        30
2.5kg      24
1.86kg     21
2.04kg     20
2.18kg     19
1.5kg      17
1.6kg      15
2.8kg      15
1.4kg      15
1.2kg      14
1.8kg      13
1.3kg      12
1.26kg     12
1.9kg      12
1.7kg      12
1.95kg     12
2.62kg     11
3kg        11
1.63kg     10
1.37kg     10
1.36kg     10
1.1kg      10
1.91kg      9
1.29kg      8
2.7kg       8
4.42kg      8
1.48kg      8
2.65kg      8
2.6kg       7
1.49kg      7
2.06kg      7
2.0kg       7
1.25kg      7
1.64kg      6
1.75kg      6
1.45kg      6
1.23kg      6
1.54kg      6
1.84kg      6
2.31kg      6
1.32kg      5
1.17kg      5
3.2kg       5
1.28kg      5
2.9kg       5
2.36kg      5
1.88kg      5
2.23kg      5
1.13kg      4
1.47kg      4
1.34kg      4
2.02kg      4
0.920kg     4
1.65kg      4
2.19kg      4
1.38kg      4
0.69kg      4
1.44kg      4
1.62kg      4
2.63kg      4
2.37kg      4
2.32kg      4
2.08kg      4
1.05kg      4
1.21kg      4
3.35kg      3
4.6kg       3

In [94]:
df.Price_euros.value_counts()

Price_euros
1099.00    12
1499.00    11
1799.00     9
899.00      9
369.00      8
999.00      8
1349.00     7
699.00      7
1199.00     7
389.00      6
629.00      6
949.00      6
799.00      6
349.00      6
549.00      6
1399.00     5
1149.00     5
299.00      5
1299.00     5
2299.00     5
459.00      5
979.00      5
1899.00     5
2799.00     4
819.00      4
1599.00     4
1195.00     4
1649.00     4
959.00      4
1749.00     4
209.00      4
479.00      4
499.00      4
649.00      4
309.00      4
689.00      4
779.00      4
2499.00     4
805.99      3
749.00      3
638.00      3
1129.00     3
839.00      3
399.00      3
449.00      3
889.00      3
398.99      3
977.00      3
1159.00     3
439.00      3
469.00      3
619.00      3
639.00      3
559.00      3
1379.00     3
249.00      3
289.00      3
599.00      3
1119.00     3
1975.00     3
579.00      3
199.00      2
919.00      2
429.00      2
720.32      2
409.00      2
1590.00     2
564.00      2
1269.00     2
597.00      2
589.00  

## Data processing

In [95]:
df.Ram = df.Ram.str.replace("GB","")
df.Ram = df.Ram.astype("int64")

In [96]:
df.Weight = df.Weight.str.replace("kg","")
df.Weight = df.Weight.astype("float64")

In [97]:
df = df.rename(columns={"Ram":"Ram (GB)","Weight":"Weight (kg)"})

In [98]:
df["Product"] = df["Product"].str.replace(r'\s*\(.*', '', regex=True)

In [99]:
df["Product"] = df["Product"].str.title()

In [100]:
df["Series"] = df["Product"].str.split(" ").str[0]

In [101]:
df.Series.value_counts()

Series
Inspiron            99
Thinkpad            72
Ideapad             69
Probook             51
Aspire              46
Elitebook           39
Latitude            35
Xps                 28
Vostro              27
Rog                 25
Yoga                25
250                 21
Zenbook             18
Macbook             17
Chromebook          15
Tecra               14
Legion              13
Vivobook            13
Alienware           12
Portege             11
Zbook               10
Satellite           10
Pavilion             8
Swift                7
V110-15Isk           7
Omen                 7
Spectre              6
Precision            6
Blade                6
Smartbook            6
Notebook             5
Envy                 5
V310-15Ikb           4
Pro                  4
Stream               4
Gl62M                4
X541Na               3
Spin                 3
Predator             3
Mi                   3
Surface              3
V330-15Ikb           3
X553Sa-Xx031T        3
Trav

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                912 non-null    int64  
 1   laptop_ID         912 non-null    int64  
 2   Company           912 non-null    object 
 3   Product           912 non-null    object 
 4   TypeName          912 non-null    object 
 5   Inches            912 non-null    float64
 6   ScreenResolution  912 non-null    object 
 7   Cpu               912 non-null    object 
 8   Ram (GB)          912 non-null    int64  
 9   Memory            912 non-null    object 
 10  Gpu               912 non-null    object 
 11  OpSys             912 non-null    object 
 12  Weight (kg)       912 non-null    float64
 13  Price_euros       912 non-null    float64
 14  Series            912 non-null    object 
dtypes: float64(3), int64(3), object(9)
memory usage: 107.0+ KB


-----------------------------------------------------------------------------------------------------------------

In [103]:
df.columns

Index(['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
       'ScreenResolution', 'Cpu', 'Ram (GB)', 'Memory', 'Gpu', 'OpSys',
       'Weight (kg)', 'Price_euros', 'Series'],
      dtype='object')

In [104]:
numeric_features = ["id","laptop_ID","Inches", "Ram (GB)", "Weight (kg)"]
categorical_features = ["Company", "Product", "TypeName", "ScreenResolution", "Cpu", "Memory", "Gpu", "OpSys", "Series"]

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)])


## Pasos que si o si debéis realizar para poder participar en la competición:

### 1. Definir X e y

In [106]:
X = df.drop(columns=["Price_euros"])
y = df["Price_euros"]

### 2. Dividir X_train, X_test, y_train, y_test

In [107]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 3. Crear y entrenar el modelo


In [118]:
xgb_pipeline = Pipeline(steps=[("preprocessor", preprocessor),
                               ("regressor", XGBRegressor(random_state=42))])

lgbm_pipeline = Pipeline(steps=[("preprocessor", preprocessor),
                                ("regressor", LGBMRegressor(random_state=42))])

catboost_pipeline = Pipeline(steps=[("preprocessor", preprocessor),
                                    ("regressor", CatBoostRegressor(random_state=42, silent=True))])

### 4. Sacar métricas, valorar el modelo 

Recuerda que en la competición se va a evaluar con la métrica de MAE.

In [119]:
xgb_scores = cross_val_score(xgb_pipeline, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
lgbm_scores = cross_val_score(lgbm_pipeline, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
catboost_scores = cross_val_score(catboost_pipeline, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')

xgb_mae = -xgb_scores.mean()
lgbm_mae = -lgbm_scores.mean()
catboost_mae = -catboost_scores.mean()

print(f'XGBRegressor MAE: {xgb_mae}')
print(f'LGBMRegressor MAE: {lgbm_mae}')
print(f'CatBoostRegressor MAE: {catboost_mae}')


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000210 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 576
[LightGBM] [Info] Number of data points in the train set: 583, number of used features: 49
[LightGBM] [Info] Start training from score 1106.809794
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

-----------------------------------------------------------------

In [124]:
best_pipeline = catboost_pipeline
best_param = {
    "regressor__iterations": [100, 200, 300, 500],
    "regressor__learning_rate": [0.01, 0.05, 0.1, 0.3],
    "regressor__depth": [3, 4, 5, 6, 7, 8]
}

random_search = RandomizedSearchCV(catboost_pipeline, best_param, n_iter=50, cv=5, scoring='neg_mean_absolute_error', random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

print(f'Best MAE: {-random_search.best_score_}')
print(f"Best MAE after RandomizedSearch: {mae}")
print(f"Best Parameters: {random_search.best_params_}")

Best MAE: 171.95786438690183
Best MAE after RandomizedSearch: 153.08997524155546
Best Parameters: {'regressor__learning_rate': 0.3, 'regressor__iterations': 500, 'regressor__depth': 5}


In [125]:
eval_data = pd.read_csv("./data/test.csv")

In [127]:
eval_data.Ram = eval_data.Ram.str.replace("GB","")
eval_data.Ram = eval_data.Ram.astype("int64")

eval_data.Weight = eval_data.Weight.str.replace("kg","")
eval_data.Weight = eval_data.Weight.astype("float64")

eval_data = eval_data.rename(columns={"Ram":"Ram (GB)","Weight":"Weight (kg)"})

eval_data["Product"] = eval_data["Product"].str.replace(r'\s*\(.*', '', regex=True)

eval_data["Product"] = eval_data["Product"].str.title()

eval_data["Series"] = eval_data["Product"].str.split(" ").str[0]

In [128]:
eval_predictions = best_model.predict(eval_data)

In [129]:

submission = pd.DataFrame({
    "id": eval_data["id"], 
    "Price_euros": eval_predictions
})

submission.to_csv("submission.csv", index=False)

In [130]:
submission.head(15)

,id,Price_euros
0,181,2363.204993
1,708,628.520290
2,862,371.605446
3,1064,1421.866080
4,702,990.021559
5,585,1496.403462
6,195,2686.074539
7,463,611.633734
8,14,3446.841444
9,621,1930.056015


## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `test.csv` nos arriesgamos a que digamos, no salga lo esperado.

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `train.csv` al completo.


**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

### 2. Carga los datos de `test.csv` para predecir.

**¿Por qué puede dar error?** 

IMPORTANTE: APLICAR A ESTOS DATOS LO MISMO QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 391 FILAS, SI O SI

**Entonces, si al cargar los datos de train usé `index_col=0` para que utilizara la primera columna del conjunto de datos como índice, ¿tendré que hacerlo también para el conjunto `test.csv`?**

In [ ]:
# ¿Qué opináis?
# Sí

![](https://www.tierraljelechu.com/web/img/wiki_up/1.996-SorpresaDto.-1-Red.jpg)

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

In [ ]:
predictions = model.predict(X_pred)
predictions

NameError: name 'model' is not defined

**¡PERO! Para subir a Kaggle la predicción, ésta tendrá que tener una forma específica y no valdrá otra.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [ ]:
sample = pd.read_csv("Dataset/sample_submission.csv") # Esta es mi ruta del archivo, la vuestra puede ser otra

In [ ]:
sample.head()

In [ ]:
sample.shape

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [ ]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("")


In [ ]:
chequeator(submission) # submission es el nombre que le habríamos puesto a nuestro .csv con los valores que me salieron en la predicción

You're ready to submit!
